# The HFM library - A fast marching solver with adaptive stencils

## Part : Application examples
## Chapter : Vehicles with trailers

We compute globally minimal paths for an omni-directional vehicle equipped with one or two trailers.
For that purpose, we numerically solve sub-Riemannian eikonal equations in dimension three and four. The numerical scheme is GPU accelerated, with features aimed to reduce numerical diffusion. The extracted trajectories are validated by comparison with a geodesic shooting method.

**References.**

The numerical scheme used in this notebook is described and studied in detail in the following publication:
* Mirebeau, J.-M. (2019). Riemannian Fast-Marching on Cartesian Grids, Using Voronoi's First Reduction of Quadratic Forms. SIAM Journal on Numerical Analysis, 57(6), 2608–2655.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Isotropic_Repro','GPU'))

In [2]:
from agd import Eikonal
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import LinearParallel as lp
norm_infinity = ad.Optimization.norm_infinity

In [3]:
import numpy as np; xp=np
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global Eikonal,ad,cugen
    Eikonal,ad,cugen = rreload([Eikonal,ad,cugen],"..")
    Eikonal.dictIn.default_mode = 'gpu'

### 0.1 Additional configuration

Uncomment the following line to use GPU acceleration.

In [5]:
xp,Eikonal,plt = [ad.cupy_friendly(e) for e in (xp,Eikonal,plt)]

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Setting dictIn.default_mode = 'gpu' in module agd.Eikonal .
Returning a copy of module matplotlib.pyplot whose functions accept cupy arrays as input.


## 1. Mathematical framework


### 1.1 The vehicle models

**Omni-directional vehicle.** Our starting point is a vehicle able to move in any direction in the plane, at unit Euclidean speed. Denote by $X \in R^2$ its position. In control theoretic terms, the evolution of $X$ is expressed in an orthonormal basis $(e,e^\perp)$ of the plane, and determined by two controls $(\alpha, \beta)$ in the unit ball.
$$
    \dot X = \alpha e + \beta e^\perp, \qquad \alpha^2 + \beta^2 \leq 1.
$$


**Case of one trailer.** Let us attach a trailer to the preceding vehicle. The trailer is rigid and has fixed length $r>0$, also denoted $r=1/\xi$. The full vehicle state is now $(X,\theta) \in R^2 \times S^1$, where $e(\theta) := (\cos \theta , \sin \theta)$ is colinear to the trailer direction and directed toward the (omni-directional) head. The trailer state cannot be controlled directly, thus the evolution of $(X,\theta)$ is again determined by two controls $(\alpha,\beta)$ in the unit ball and reads
$$
    \begin{pmatrix}
    \dot X\\
    \dot \theta
    \end{pmatrix}
    =
    \alpha 
    \begin{pmatrix}
    e(\theta)\\
    0
    \end{pmatrix}
    {+}
    \beta
    \begin{pmatrix}
    e(\theta)^\perp\\
    \xi
    \end{pmatrix}.
$$
Note that we use a block notation for vectors; for instance $(X,\theta) := (X_0,X_1,\theta)$ if $X = (X_0,X_1) \in R^2$.
A few remarks are in order:
- The head $X$ of the vehicle follows the same dynamics as in the previous simpler model, only expressed in the peculiar orthonormal basis $e((\theta),e(\theta)^\perp)$. Again, the head can always move in any direction, at unit Euclidean speed.
- If the head moves in the direction $e(\theta)$, aligned with the trailer direction, then the latter is only trailing behing and its orientation does not change, as expected.
- A special motion of interest is when the head moves in circles of radius $r = 1/\xi$ in the direction $e(\theta)^\perp$, so that the rear wheels of the trailer do not move, although its angle $\theta$ changes continuously. The second control vector corresponds, at first order, to this motion.

**Case of two trailers.**
Let us attach a second trailer at to the preceding vehicle, at the back of the first trailer. The additional trailer is again rigid, and has fixed length $r'>0$, also denoted $r' = 1/\zeta$. The angle of the second trailer is again denoted $\phi$. The vehicle state $(X,\theta,\phi) \in R^2 \times S^1 \times S^1$ is now four dimensional, but still only two controls $(\alpha, \beta)$ are available. The evolution equation reads:
$$
    \begin{pmatrix}
    \dot X \\
    \dot \theta \\
    \dot \phi
    \end{pmatrix}
    =
    \alpha 
    \begin{pmatrix}
    e(\theta) \\
    0 \\
    \zeta \sin(\theta-\phi)
    \end{pmatrix}
    {+}
    \beta
    \begin{pmatrix}
    e(\theta)^\perp \\
    \xi \\
    0
    \end{pmatrix}
$$
- Control $\alpha$ corresponds to the vehicle moving forward, so that the first trailer orientation is not affected, but the second trailer tends to align with the first one.
- Control $\beta$ corresponds to the omni-directional head moving into circles of radius $\xi$, leaving in place the rear wheels of the first trailer, as described above. The second trailer is unaffected by this motion.

### 1.2 Properties of the models

**Commutators.**
Denote by $\omega_0$ and $\omega_1$ the two control vector fields of (one of our three) models. In the trailer-less case, $\omega_0$ and $\omega_1$ span the plane $R^2$. In other words, any tangent vector to the state space $R^2$ can be realized by a suitable linear combination $\alpha\omega_0+ \beta \omega_1$.

As soon as one trailer is attached, the two vectors vector fields $\omega_0,\omega_1$ fail to span the tangent space to the state space $R^2 \times S^1$, which is three dimensional. The model is said non-holonomic. Alternating small (first-order) displacements along $\omega_0$ and $\omega_1$ one may however realize a motion (second-order small) in a direction $\omega_2$ which is not a linear combination of $\omega_0$ and $\omega_1$, namely here $\omega_2 = (e(\theta)^\perp,0)$. One says that $\omega_2$ is the commutor of $\omega_0$ and $\omega_1$.

When two trailers are attached, the two vector fields $\omega_0, \omega_1$ and their first order commutator $\omega_2$ still do not span the tangent space to the state space $R^2 \times S^1 \times S^1$, which is four dimensional. Yet another commutator, of $\omega_2$ with $\omega_0$ or $\omega_1$ is required for that purpose, and one says that the model is sub-Riemannian of rank two.


**Comparison with the Reeds-Shepp model.**

To (some) mathematicians, wether the dog is wagging the tail, or the tail is wagging the dog, is only a question of coordinate system choice. Following that rationale, and we denote by $Y = X - e(\theta)/\xi$ the position of the tail of the trailer, in the single trailer case. One easily checks that 
$$
    \begin{pmatrix}
    \dot Y \\
    \dot \theta 
    \end{pmatrix}
    = 
    \alpha
    \begin{pmatrix}
    e(\theta) \\
    0
    \end{pmatrix}
    {+}
    \beta
    \begin{pmatrix}
    0 \\
    \xi
    \end{pmatrix}.
$$
This particular optimal control model is known as the Reeds-Shepp car, and best represents a wheelchair-like vehicle. Up to a coordinate change, it is thus isomorphic to the omni-directional vehicle with a single trailer. 
(Implementation-wise, one notable difference is that the the Reeds-Shepp hamiltonian is block diagonal with sizes (2,1), in contrast with the original model which has no such structure.)

The omni-directional vehicle with two trailers is, likewise, isomorphic to a Reeds-Shepp car with a single trailer.

### 1.3 Sub-Riemannian formalism

**Optimal control.** Denote by $\Omega$ a sub-domain of the state space (for our models $R^2$, $R^2 \times S^1$ or $R^2 \times S^1 \times S^1$). 
Denote by $\omega_0, \cdots, \omega_n$ the control vector fields (for our models $n=1$), and by $\alpha_0, \cdots, \alpha_n$ the related controls (for our models denoted $\alpha, \beta$).

Our objective is to evaluate the minimal cost $U(z)$ of a path from a reference seed point $z_0$ to a given point $z$. More precisely, 
$$
    U(z)^2 = \min_{\alpha_0,\cdots,\alpha_n}\int_0^1 (\alpha_0(t)^2 +\cdots+ \alpha_n(t)^2) dt,
$$
subject to the following constraint: one has $Z(1) = z$, where $Z : [0,1] \to \Omega$ is the path characterized by $Z(0) = z_0$ and for all $0 \leq t \leq 1$
$$
    \dot Z(t) = \alpha_0(t) \omega_0(Z(t)) + \cdots + \alpha_n(t) \omega_n(Z(t)).
$$

**Eikonal equation.** The value function $U : \Omega \to R$ is, under suitable assumptions, the unique viscosity solution of the sub-Riemannian eikonal equation. More precisely, one has $U(z_0) = 0$, outflow boundary conditions on $\partial \Omega$, and for all $z \in \Omega$ 
$$
    \|\nabla u(z)\|_{D(z)} = 1,
$$
where $D(z) = \omega_0(z) \omega_0(z)^T + \cdots + \omega_n(z) \omega_n(z)^T$ is the sum of the self outer-products of the control vector fields.
Note that $D(z)$ is a positive semi-definite matrix, which is rank deficient unless $(\omega_0(z), \cdots,\omega_n(z))$ span the tangent space. The function $H(q,p) := \frac 1 2 <p,D(q)p>$ is referred to as the Hamiltonian of the system.

**Riemannian relaxation.**
Denote by $\tilde \omega_0, \cdots , \tilde \omega_{\tilde n}$ a family of vector fields which, together with $\omega_0, \cdots, \omega_n$, span the tangent space to the state space. Let also $\tilde D := \tilde\omega_0(z) \tilde\omega_0(z)^T + \cdots + \tilde\omega_{\tilde n}(z) \cdots \tilde \omega_{\tilde n}(z)^T$, and for any $\epsilon >0$
$$
    D_\epsilon(z) := D(z) + \epsilon^2 \tilde D(z).
$$
Then $D_\epsilon(z)$ is a positive definite (full rank) matrix, and the Riemannian eikonal equation 
$$
    \|\nabla u_\epsilon(z)\|_{D_\epsilon(z)} = 1,
$$
is referred to as the $\epsilon$-relaxation of the sub-Riemannian eikonal equation. The Riemannian eikonal equation can be discretized using the techniques which are standard in this series of notebooks, see below. For sufficiently small $\epsilon>0$, the solution $u_\epsilon$ to the Riemannian relaxation is close to the solution $u$ of the original sub-Riemannian equation, see the reference.

<!---
### 1.2 Sub-Riemannian formalism

**Describing a sub-Riemannian model.**

Sub-Riemannian geometry is a limit case of Riemannian geometry, where the cost of motion in certain directions tends to infinity. Heuristically, one could say that some of the eigenvalues of the Riemannian metric tensor tend to infinity.
A simpler and more rigorous approach is to consider the matrix of the Hamiltonian, defined as the inverse of the Riemannian tensor, which in the sub-Riemannian case is rank-deficient.

Our numerical approach to solving eikonal equations does however require a positive definite Hamiltonian. We thus introduce a relaxation parameter $\epsilon>0$ and consider 
$$
    D(x) = D_0(x) + \epsilon^2 D_1(x),
$$
where $D_0$ is the rank deficient hamiltonian of interest, and $D_1$ is introduced so that the sum $D$ is positive definite. By assumption, $D_0$ and $D_1$ must be positive semi-definite.
If $\epsilon$ is small, then the Riemannian eikonal equation based on the matrix $D$, 
$$
\|\nabla u(x) \|_{D(x)} = 1
$$
is a good enough approximation of the sub-Riemannian eikonal equation, involving $D_0$ instead. We denoted $\|v\|_D := \sqrt{<v,Dv>}$.
--->

### 1.4 Numerical implementation

**Discretization of the eikonal equation.**
Assume that a tensor field $D$ admits a decomposition of the type
$$
    D = \sum_{i \in I} \lambda_i e_i e_i^T,
$$
where the index set $I$ is finite, $\lambda_i \geq 0$ is a non-negative weight, and $e_i \in Z^d$ is an offset with integer coordinates, for each $i \in I$.
The pointwise dependency of $D=D(x)$, $\lambda_i=\lambda_i(x)$, $e_i=e_i(x)$ is omitted for readability.
We discretize the Riemannian eikonal equation based on the following finite differences approximation:
$$
    \|\nabla u(x)\|_{D(x)}^2 \approx h^{-2} \sum_{i \in I} \max\{0,u(x)-u(x-h e_i), u(x)-u(x+h e_i)\}^2.
$$
This resulting numerical scheme enjoys properties (monotony, causality) which allow to establish the convergence and stability of the method, and to implement it efficiently, see the reference above.

**Tensor decomposition techniques and their limitations.**
[Selling's](../Notebooks_Algo/TensorSelling.ipynb) and [Voronoi's](../Notebooks_Algo/TensorVoronoi.ipynb) matrix decompositions, are our main tools for obtaining the tensor decomposition presented above. They do require however, that the matrices $D(z)$, $z \in \Omega$, be positive definite. In fact, no such decomposition exists for semi-definite matrices in general.

In the case of a sub-Riemannian eikonal equation, involving a rank deficient tensor field $D$, we thus apply the previous approach to the Riemannian relaxation $D_\epsilon$.

<!---
**Numerical diffusion and pruning.**
Consider a decomposition of a Riemannian relaxation $D_\epsilon = D + \epsilon^2 \tilde D$, and note that the Frobenius norm of $D_\epsilon$ can be expressed as follows:
$$
    \mathrm{Tr}(D_\epsilon^2) = \sum_{i \in I} \lambda_i (\|e_i\|_D^2 + \epsilon^2 \| e_i \|_{\tilde D}^2),
$$
where all the contributions are non-negative.
The pruning step eliminates from the numerical scheme all offsets such that $\|e_j\|_D \leq \epsilon \|e_j\|_{\tilde D}$, which helps moderate the numerical diffusion, while maintaining the consistency with the sub-Riemannian eikonal equation as $\epsilon \to 0$.
--->

## 2. One trailer

In [6]:
θs = np.linspace(0,2*np.pi,96)

### 2.1 Control vector fields

We reproduce the two control vector fields defining the model, whose mathematical expression is given in the first section.

In [41]:
def ω(θ,ξ=1):
    z,u = np.zeros_like(θ), np.ones_like(θ)
    return np.stack(ad.asarray([
        [ np.cos(θ),np.sin(θ),  z],
        [-np.sin(θ),np.cos(θ),ξ*u]
    ]),axis=1)

We also introduce one additional control vector field, used in the Riemannian relaxation, which together with the first two spans the tangent space.

In [42]:
def ω_(θ,ξ=1):
    u = np.ones_like(θ)
    return np.stack(ad.asarray([
        [ξ*np.sin(θ),-ξ*np.cos(θ),u]
    ]),axis=1)

The additional vector field is orthogonal to the two basic fields. This is a common choice, although it is not required.

<!---
We now implement the sum $D = \omega_0\omega_0^T+ \cdots+ \omega_n \omega_n^T$ of the self outer products of some vectors.
As mentioned in the introduction, this is a semi-definite matrix, which is rank deficient unless the vectors $(\omega_0,\cdots,\omega_n)$ span $R^d$. 

If $\omega_1,\cdots,\omega_n$ are the control vector fields, then $D$ defines the sub-Riemannian hamiltonian. In the full-rank Riemannian case, it is also the matrix inverse, or dual, of the metric.

def ω2D(ω): 
    """
    DUAL of the (sub-)Riemannian metric 
    defined by a family of control vector fields.
    """
    return Metrics.Riemann(lp.outer_self(ω).sum(axis=1))
--->

<!---
We choose not to store this data in the `Metrics.Riemann` class, since it is 
**Important note.**
In most notebooks, the *Metrics.Riemann* class is used to store a
The next function evaluates the sum $\omega_0\omega_0^T+ \cdots+ \omega_n \omega_n^T$
--->

In [43]:
assert np.allclose(lp.dot_VV(ω(θs),ω_(θs)),0)

In [44]:
def Hamiltonian(controls,complement,eps=0.1):
    return lp.outer_self(controls).sum(axis=2) + lp.outer_self(eps*complement).sum(axis=2) 

In [45]:
H = Hamiltonian(ω(θs),ω_(θs))
print(f"Shape of the dual metric {H.shape}")
print(f"Eigenvalues at an arbitrary point {np.linalg.eigvalsh(H[:,:,10])}")

Shape of the dual metric (3, 3, 96)
Eigenvalues at an arbitrary point [ 0.02     1     2]


### 2.2 Input

In [46]:
hfmIn = Eikonal.dictIn({
    'model':'Riemann3',
    'seed':[0,0,0],
    'periodic':[False,False,True],
})
hfmIn.SetRect([[-1,1],[-1,1],[-np.pi,np.pi]],dims = [101,101,96])
hfmIn['origin'][2] -= hfmIn['gridScales'][2]/2 # Angular grid starts at -pi
hfmIn.SetUniformTips([4,4,4])

Our metric is translation invariant w.r.t the first two coordinates $X = (X_0,X_1)$, only depending on the *last coordinate* the angle $\theta$. This structure is relatively common, and the HFM library will take advantage of it to save some memory and compute less stencils.

More generally, metrics depending only on some of the final coordinates $(x_k,...,x_n)$ are implicitly regarded invariant w.r.t the first coordinates, see also the two trailer case in the next section. (This is a form of broadcasting.)

In [47]:
_,_,aθ = hfmIn.Axes()
assert np.allclose(aθ[0],-np.pi)

In [48]:
ξ=1.
hfmIn.update({
    'dualMetric':Metrics.Riemann(Hamiltonian(ω(aθ,ξ), ω_(aθ,ξ))),
})

In [49]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
DEBUG _SetGeometry.py  True (96,) 2 (6, 24, 4)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
_Kernel :  4   24
DEBUG _kernel.py (24, 4, 6, 3) float32
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.2330014705657959 seconds, and 96 iterations.
Post-Processing


ValueError: Unsupported trait periodic_axes:([False, False, True], [False, False, True], [False, False, True])

## 3. Two trailers

In [59]:
angles = np.linspace(0,2*np.pi,96)
θs,ϕs = np.meshgrid(angles,angles,indexing='ij')

### 3.1 Control vector fields

Again, we reproduce the two control vector fields defining the model, whose mathematical expression is given in the first section.

In [61]:
def ω(θ,ϕ,ξ=1,ζ=1):
    z,u = np.zeros_like(θ), np.ones_like(θ)
    return np.stack([
        [ np.cos(θ),np.sin(θ),z,ζ*np.sin(θ-ϕ)],
        [-np.sin(θ),np.cos(θ),ξ*u,z]
    ],axis=1)

We also introduce two additional control vector fields, used in the Riemannian relaxation, which together with the first two span the tangent space.

In [62]:
def ω_(θ,ϕ,ξ=1,ζ=1):
    z,u = np.zeros_like(θ), np.ones_like(θ)
    ρ = -ζ*np.sin(θ-ϕ)
    return np.stack([
        [ρ*np.cos(θ), ρ*np.sin(θ),z,u],
        [ξ*np.sin(θ),-ξ*np.cos(θ),u,z]
    ],axis=1)

In [63]:
assert np.allclose(lp.dot_VV(ω(θs,ϕs),ω_(θs,ϕs)),0)

### 3.2 Input

In [57]:
hfmIn = Eikonal.dictIn({
    'model':'SubRiemann4',
    'seed':[0,0,0,0],
    'periodic':[False,False,True,True],
})
hfmIn.SetRect([[-0.5,0.5],[-0.5,0.5],[-np.pi,np.pi],[-np.pi,np.pi]],dims=[51,51,64,64])
hfmIn['origin'][2:] -= hfmIn['gridScales'][2:]/2 # Angular grid starts at -pi
hfmIn.SetUniformTips([4,4,4,3])

Our metric is translation invariant w.r.t the first two coordinates $X = (X_0,X_1)$, only depending on the *last two coordinates* the angle $\theta$ and $\phi$. This structure is relatively common, and the HFM library will take advantage of it to save some memory and compute less stencils.

In [59]:
_,_,aθ,aϕ = hfmIn.Axes()
assert np.allclose(aθ[0],-np.pi) and np.allclose(aϕ[0],-np.pi)
θs,ϕs = np.meshgrid(aθ,aϕ,indexing='ij')

In [64]:
ξ,ζ=1.,1.
hfmIn.update({
    'controls':[ω(θs,ϕs,ξ,ζ), ω_(θs,ϕs,ξ,ζ)],
    'eps':0.1,
})